In [31]:

import scipy
import numpy
import pandas

from scipy import stats

In [3]:
from new_base import Path, Projector, Item, FoldGenerator

In [6]:
from new_base_trueuse_pods import features, path_pseudo_edges, path_matrix, path_vertices, sources, name_list, param_list
from functional import SomersD, pv_metric

n_folds = 10
joint_lag = 12
val_rate = 0.5
overlap_rate = 0.15
fg = FoldGenerator(n_folds=n_folds, joint_lag=joint_lag, val_rate=val_rate, overlap_rate=overlap_rate)

target = 'TLT_aggmean_pct'

fg.init_path(path_vertices, path_matrix, path_pseudo_edges)

timeaxis = sources[1].series['DATE'].values[sources[1].series['DATE'].values >= pandas.to_datetime('2007-01-01').isoformat()]

projector = Projector
# performer = SomersD
performer = pv_metric
target_source = 'TLT'
target_transform = path_pseudo_edges[path_vertices.index(target)]


In [8]:

j = 0

x_factors_in = list(features)
name = x_factors_in[j]
print(name)

GS1


In [16]:
fg_local = fg.copy()

sub_features = ['{0}__{1}'.format(name, name_list[j]) for j in range(len(name_list))]

target_components = [x for x in fg.path.path_vertices if target_source in x]
target_components_mask = numpy.isin(fg.path.path_vertices, target_components)
target_components_ix = numpy.arange(fg.path.path_matrix.shape[0])[target_components_mask]
path_matrix_sub_target = fg.path.path_matrix[target_components_ix[:, numpy.newaxis], target_components_ix]
path_vertices_sub_target = numpy.array(fg.path.path_vertices)[target_components_ix].tolist()
path_pseudo_edges_sub_target = fg.path.path_pseudo_edges[target_components_ix].tolist()
n_targets = target_components_mask.sum()

local_path_vertices = path_vertices_sub_target + [name] + sub_features
local_path_matrix = numpy.zeros(shape=((len(name_list) + 1 + n_targets), (len(name_list) + 1 + n_targets)))

targets_mask = numpy.arange(n_targets)
local_path_matrix[targets_mask[:, numpy.newaxis], targets_mask] = path_matrix_sub_target

local_path_matrix[n_targets, 3:] = 1

local_path_pseudo_edges = path_pseudo_edges_sub_target + [None] + [projector(**param_list[j]) for j in range(len(param_list))]


fg_local.init_path(local_path_vertices, local_path_matrix, local_path_pseudo_edges)


In [20]:
len(local_path_vertices), local_path_matrix.shape, len(local_path_pseudo_edges)

(57, (57, 57), 57)

In [ ]:
local_path_vertices

In [23]:

fold_n = 1
print(fold_n)

local_sources = [x for x in sources if (x.name == target_source) or (x.name == name)]
data_train, data_test = fg_local.fold(local_sources, sub_features + [name] + [target], timeaxis, fold_n=fold_n)
x_train, y_train = data_train[[x for x in data_train.columns if x != target]].iloc[:-1, :], data_train[target].iloc[1:]
x_test, y_test = data_test[[x for x in data_test.columns if x != target]].iloc[:-1, :], data_test[target].iloc[1:]

local_resulted = []
for j in range(len(name_list)):

    performed = performer(x=x_test.iloc[:, j].values, y=y_test.values)
    local_resulted.append([fold_n, name_list[j], performed, numpy.nan])
local_resulted = pandas.DataFrame(data=local_resulted, columns=['fold_n', 'transform', 'performed', 'stabilized'])

base_performed = performer(x=x_test.iloc[:, len(name_list)].values, y=y_test.values)
local_resulted['base_performed'] = numpy.abs(base_performed)

# xxl = pandas.DataFrame(data={'x': x_test.iloc[:, len(self.name_list)].values, 'y': y_test.values})


1
Folding: 1 / 10 

	lag_start=	2007-12-01T00:00:00+00:00; 
	start=		2008-12-01T00:00:00+00:00; 
	lag_mid=	2011-04-01T00:00:00+00:00; 
	mid=		2012-04-01T00:00:00+00:00; 
	end=		2015-08-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	40


C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\env\macro_new\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
C:\TET\env\macro_new\Lib\site-packages\sklearn\preproces

In [ ]:
# GS1__p101x3_div_pct
# ...
# ..

# GS1__pct_shift1

x_train['GS1__pct_shift1'].values, x_train[target].values

In [28]:
local_resulted

,fold_n,transform,performed,stabilized,base_performed
0,1,mean1x3_div_pct,0.113308,NaN,0.063989
1,1,mean1x6_div_pct,0.263196,NaN,0.063989
2,1,mean1x12_div_pct,0.371224,NaN,0.063989
3,1,mean3x6_div_pct,0.320055,NaN,0.063989
4,1,mean3x12_div_pct,0.409035,NaN,0.063989
5,1,pct_shift1,0.434802,NaN,0.063989
6,1,pct_shift3,0.269584,NaN,0.063989
7,1,pct_shift6,0.067957,NaN,0.063989
8,1,pct_shift12,-0.135259,NaN,0.063989
9,1,median1x3_div_pct,0.207938,NaN,0.063989


In [24]:
data_train

,GS1__mean1x3_div_pct,GS1__mean1x6_div_pct,GS1__mean1x12_div_pct,GS1__mean3x6_div_pct,GS1__mean3x12_div_pct,GS1__pct_shift1,GS1__pct_shift3,GS1__pct_shift6,GS1__pct_shift12,GS1__median1x3_div_pct,...,GS1__stayer_mimx,GS1__stayer_percentile,GS1__binner_mimx5,GS1__binner_percentile5,GS1__binner_mimx10,GS1__binner_percentile10,GS1__binner_mimx20,GS1__binner_percentile20,GS1,TLT_aggmean_pct
2008-12-01T00:00:00+00:00,-0.019608,-0.117647,-0.176471,-0.098039,-0.156863,-0.781009,-1.360453,-1.597117,-1.895077,-0.039216,...,0.000000,0.019608,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.49,0.169617
2009-01-01T00:00:00+00:00,0.313725,0.215686,0.137255,-0.098039,-0.176471,-0.107631,-1.171637,-1.645156,-1.817929,0.254902,...,0.599116,0.352941,0.414007,0.2,0.509503,0.3,0.533802,0.30,0.44,-0.041724
2009-02-01T00:00:00+00:00,0.823529,0.862745,0.647059,0.039216,-0.176471,0.342945,-0.545694,-1.257361,-1.195876,0.647059,...,1.000000,1.000000,1.000000,0.8,1.000000,0.9,1.000000,0.90,0.62,-0.069847
2009-03-01T00:00:00+00:00,-0.078431,0.529412,0.274510,0.607843,0.352941,0.031749,0.267063,-1.093390,-0.878069,0.000000,...,0.723124,0.705882,0.722307,0.6,0.767546,0.7,0.732565,0.65,0.64,-0.003796
2009-04-01T00:00:00+00:00,-0.529412,0.019608,-0.156863,0.549020,0.372549,-0.151550,0.223144,-0.948494,-1.151722,-0.490196,...,0.560040,0.215686,0.414007,0.2,0.457271,0.2,0.488372,0.20,0.55,-0.011107
2009-05-01T00:00:00+00:00,-0.039216,0.117647,0.098039,0.156863,0.137255,-0.095310,-0.215111,-0.760806,-1.415853,0.000000,...,0.610077,0.392157,0.414007,0.2,0.509503,0.3,0.548362,0.35,0.50,-0.067367
2009-06-01T00:00:00+00:00,0.254902,0.019608,0.411765,-0.235294,0.156863,0.019803,-0.227057,0.040005,-1.557112,0.294118,...,0.712495,0.686275,0.722307,0.6,0.673803,0.6,0.672771,0.60,0.51,-0.041891
2009-07-01T00:00:00+00:00,-0.058824,-0.196078,0.196078,-0.137255,0.254902,-0.060625,-0.136132,0.087011,-1.558144,0.000000,...,0.640938,0.470588,0.534648,0.4,0.556649,0.4,0.593724,0.45,0.48,0.017608
2009-08-01T00:00:00+00:00,-0.019608,0.039216,0.294118,0.058824,0.313725,-0.042560,-0.083382,-0.298493,-1.555854,0.000000,...,0.657010,0.568627,0.534648,0.4,0.595831,0.5,0.617435,0.55,0.46,0.006059
2009-09-01T00:00:00+00:00,-0.156863,-0.156863,-0.019608,0.000000,0.137255,-0.139762,-0.242946,-0.470004,-1.563394,-0.215686,...,0.570528,0.254902,0.414007,0.2,0.457271,0.2,0.488372,0.20,0.40,0.027499


In [25]:
data_test

,GS1__mean1x3_div_pct,GS1__mean1x6_div_pct,GS1__mean1x12_div_pct,GS1__mean3x6_div_pct,GS1__mean3x12_div_pct,GS1__pct_shift1,GS1__pct_shift3,GS1__pct_shift6,GS1__pct_shift12,GS1__median1x3_div_pct,...,GS1__stayer_mimx,GS1__stayer_percentile,GS1__binner_mimx5,GS1__binner_percentile5,GS1__binner_mimx10,GS1__binner_percentile10,GS1__binner_mimx20,GS1__binner_percentile20,GS1,TLT_aggmean_pct
2012-04-01T00:00:00+00:00,-0.333333,-0.215686,-0.058824,0.117647,0.274510,-0.052632,0.500000,0.636364,-0.280000,-0.411765,...,0.646772,0.529412,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,0.18,0.017699
2012-05-01T00:00:00+00:00,0.000000,-0.058824,0.058824,-0.058824,0.058824,0.055555,0.187500,0.727273,0.000000,0.000000,...,0.742981,0.725490,0.054067,0.054067,0.054067,0.054067,0.054067,0.054067,0.19,0.047945
2012-06-01T00:00:00+00:00,0.049020,-0.107843,-0.029412,-0.156863,-0.078431,0.000000,0.000000,0.583333,0.055555,0.000000,...,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.19,0.041378
2012-07-01T00:00:00+00:00,-0.029412,-0.107843,-0.029412,-0.078431,0.000000,0.000000,0.055555,0.583333,0.000000,0.000000,...,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.19,0.020485
2012-08-01T00:00:00+00:00,-0.058824,-0.137255,-0.176471,-0.078431,-0.117647,-0.052632,-0.052632,0.125000,0.636364,-0.107843,...,0.646772,0.529412,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,0.18,-0.029612
2012-09-01T00:00:00+00:00,0.049020,0.049020,-0.068627,0.000000,-0.117647,0.000000,-0.052632,-0.052632,0.800000,0.000000,...,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18,-0.017255
2012-10-01T00:00:00+00:00,0.049020,0.049020,-0.068627,0.000000,-0.117647,0.000000,-0.052632,0.000000,0.636364,0.000000,...,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18,-0.003977
2012-11-01T00:00:00+00:00,0.000000,0.049020,-0.068627,0.049020,-0.068627,0.000000,0.000000,-0.052632,0.636364,0.000000,...,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18,0.019247
2012-12-01T00:00:00+00:00,-0.274510,-0.294118,-0.392157,-0.019608,-0.117647,-0.111111,-0.111111,-0.157895,0.333333,-0.343137,...,0.590083,0.294118,-0.117783,-0.117783,-0.117783,-0.117783,-0.117783,-0.117783,0.16,-0.014223
2013-01-01T00:00:00+00:00,0.000000,-0.117647,-0.215686,-0.117647,-0.215686,-0.062500,-0.166667,-0.210526,0.250000,0.000000,...,0.637455,0.450980,-0.064538,-0.064538,-0.064538,-0.064538,-0.064538,-0.064538,0.15,-0.033629


In [26]:
x_test

,GS1__mean1x3_div_pct,GS1__mean1x6_div_pct,GS1__mean1x12_div_pct,GS1__mean3x6_div_pct,GS1__mean3x12_div_pct,GS1__pct_shift1,GS1__pct_shift3,GS1__pct_shift6,GS1__pct_shift12,GS1__median1x3_div_pct,...,GS1__mean6x12_quant_pct,GS1__stayer_mimx,GS1__stayer_percentile,GS1__binner_mimx5,GS1__binner_percentile5,GS1__binner_mimx10,GS1__binner_percentile10,GS1__binner_mimx20,GS1__binner_percentile20,GS1
2012-04-01T00:00:00+00:00,-0.333333,-0.215686,-0.058824,0.117647,0.274510,-0.052632,0.500000,0.636364,-0.280000,-0.411765,...,0.166667,0.646772,0.529412,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,0.18
2012-05-01T00:00:00+00:00,0.000000,-0.058824,0.058824,-0.058824,0.058824,0.055555,0.187500,0.727273,0.000000,0.000000,...,0.166667,0.742981,0.725490,0.054067,0.054067,0.054067,0.054067,0.054067,0.054067,0.19
2012-06-01T00:00:00+00:00,0.049020,-0.107843,-0.029412,-0.156863,-0.078431,0.000000,0.000000,0.583333,0.055555,0.000000,...,0.166667,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.19
2012-07-01T00:00:00+00:00,-0.029412,-0.107843,-0.029412,-0.078431,0.000000,0.000000,0.055555,0.583333,0.000000,0.000000,...,0.166667,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.19
2012-08-01T00:00:00+00:00,-0.058824,-0.137255,-0.176471,-0.078431,-0.117647,-0.052632,-0.052632,0.125000,0.636364,-0.107843,...,0.666667,0.646772,0.529412,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,-0.054067,0.18
2012-09-01T00:00:00+00:00,0.049020,0.049020,-0.068627,0.000000,-0.117647,0.000000,-0.052632,-0.052632,0.800000,0.000000,...,0.833333,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18
2012-10-01T00:00:00+00:00,0.049020,0.049020,-0.068627,0.000000,-0.117647,0.000000,-0.052632,0.000000,0.636364,0.000000,...,0.750000,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18
2012-11-01T00:00:00+00:00,0.000000,0.049020,-0.068627,0.049020,-0.068627,0.000000,0.000000,-0.052632,0.636364,0.000000,...,0.833333,0.694876,0.637255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18
2012-12-01T00:00:00+00:00,-0.274510,-0.294118,-0.392157,-0.019608,-0.117647,-0.111111,-0.111111,-0.157895,0.333333,-0.343137,...,0.833333,0.590083,0.294118,-0.117783,-0.117783,-0.117783,-0.117783,-0.117783,-0.117783,0.16
2013-01-01T00:00:00+00:00,0.000000,-0.117647,-0.215686,-0.117647,-0.215686,-0.062500,-0.166667,-0.210526,0.250000,0.000000,...,0.833333,0.637455,0.450980,-0.064538,-0.064538,-0.064538,-0.064538,-0.064538,-0.064538,0.15


In [27]:
y_test

2012-05-01T00:00:00+00:00    0.047945
2012-06-01T00:00:00+00:00    0.041378
2012-07-01T00:00:00+00:00    0.020485
2012-08-01T00:00:00+00:00   -0.029612
2012-09-01T00:00:00+00:00   -0.017255
2012-10-01T00:00:00+00:00   -0.003977
2012-11-01T00:00:00+00:00    0.019247
2012-12-01T00:00:00+00:00   -0.014223
2013-01-01T00:00:00+00:00   -0.033629
2013-02-01T00:00:00+00:00   -0.016010
2013-03-01T00:00:00+00:00   -0.002083
2013-04-01T00:00:00+00:00    0.042303
2013-05-01T00:00:00+00:00   -0.029431
2013-06-01T00:00:00+00:00   -0.051081
2013-07-01T00:00:00+00:00   -0.034799
2013-08-01T00:00:00+00:00   -0.028541
2013-09-01T00:00:00+00:00   -0.006183
2013-10-01T00:00:00+00:00    0.020192
2013-11-01T00:00:00+00:00   -0.019849
2013-12-01T00:00:00+00:00   -0.014363
2014-01-01T00:00:00+00:00    0.020816
2014-02-01T00:00:00+00:00    0.019447
2014-03-01T00:00:00+00:00    0.007603
2014-04-01T00:00:00+00:00    0.018361
2014-05-01T00:00:00+00:00    0.025009
2014-06-01T00:00:00+00:00   -0.005350
2014-07-01T0

In [37]:
global_resulted = None

for fold_n in fg_local.folds:
    
    local_sources = [x for x in sources if (x.name == target_source) or (x.name == name)]
    data_train, data_test = fg_local.fold(local_sources, sub_features + [name] + [target], timeaxis, fold_n=fold_n)
    x_train, y_train = data_train[[x for x in data_train.columns if x != target]].iloc[:-1, :], data_train[target].iloc[1:]
    x_test, y_test = data_test[[x for x in data_test.columns if x != target]].iloc[:-1, :], data_test[target].iloc[1:]
    
    local_resulted = []
    for j in range(len(name_list)):
    
        performed = performer(x=x_test.iloc[:, j].values, y=y_test.values)
        local_resulted.append([fold_n, name_list[j], performed, numpy.nan])
    local_resulted = pandas.DataFrame(data=local_resulted, columns=['fold_n', 'transform', 'performed', 'stabilized'])
    
    base_performed = performer(x=x_test.iloc[:, len(name_list)].values, y=y_test.values)
    local_resulted['base_performed'] = base_performed
    
    # xxl = pandas.DataFrame(data={'x': x_test.iloc[:, len(self.name_list)].values, 'y': y_test.values})
    
    if global_resulted is None:
        global_resulted = local_resulted.copy()
    else:
        global_resulted = pandas.concat((global_resulted, local_resulted), axis=0, ignore_index=False)

Folding: 0 / 10 

	lag_start=	2007-01-01T00:00:00+00:00; 
	start=		2008-01-01T00:00:00+00:00; 
	lag_mid=	2010-04-01T00:00:00+00:00; 
	mid=		2011-04-01T00:00:00+00:00; 
	end=		2014-08-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	40


C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)


Folding: 1 / 10 

	lag_start=	2007-12-01T00:00:00+00:00; 
	start=		2008-12-01T00:00:00+00:00; 
	lag_mid=	2011-04-01T00:00:00+00:00; 
	mid=		2012-04-01T00:00:00+00:00; 
	end=		2015-08-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	40


C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\env\macro_new\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
C:\TET\env\macro_new\Lib\site-packages\sklearn\preproces

Folding: 2 / 10 

	lag_start=	2008-12-01T00:00:00+00:00; 
	start=		2009-12-01T00:00:00+00:00; 
	lag_mid=	2012-04-01T00:00:00+00:00; 
	mid=		2013-04-01T00:00:00+00:00; 
	end=		2016-08-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	40


...

C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\env\macro_new\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
C:\TET\env\macro_new\Lib\site-packages\sklearn\preproces

Folding: 8 / 10 

	lag_start=	2014-12-01T00:00:00+00:00; 
	start=		2015-12-01T00:00:00+00:00; 
	lag_mid=	2018-04-01T00:00:00+00:00; 
	mid=		2019-04-01T00:00:00+00:00; 
	end=		2022-08-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	40


C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)


Folding: 9 / 10 

	lag_start=	2015-12-01T00:00:00+00:00; 
	start=		2016-12-01T00:00:00+00:00; 
	lag_mid=	2019-04-01T00:00:00+00:00; 
	mid=		2020-04-01T00:00:00+00:00; 
	end=		2023-07-01T00:00:00+00:00; 
	val_rate=	0.50
	size=	39


C:\TET\macro\new_base_test_projectors.py:450: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)
C:\TET\macro\new_base_test_projectors.py:456: FutureWarning: The provided callable <function mean at 0x000001E539973420> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  result = s.reset_index().groupby(by='index').agg(self.func)


In [38]:

# adjust for potential change of the signs
n_performed = (global_resulted['performed'] >= 0).sum()
if n_performed < (global_resulted['performed'].shape[0] / 2):
    global_resulted['performed'] = global_resulted['performed'] * (-1)
n_base = (global_resulted['base_performed'] >= 0).sum()
if n_base < (global_resulted['base_performed'].shape[0] / 2):
    global_resulted['base_performed'] = global_resulted['base_performed'] * (-1)

# https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/paired-sample-t-test/
global_resulted['perf_diff'] = global_resulted['performed'] - global_resulted['base_performed']
global_resulted_agg_part = global_resulted.groupby(by='transform')
global_resulted_agg_mean = global_resulted_agg_part[['perf_diff']].mean().rename(columns={'perf_diff': 'mean'})
global_resulted_agg_std = global_resulted_agg_part[['perf_diff']].std().rename(columns={'perf_diff': 'std'})
global_resulted_agg_count = global_resulted_agg_part[['perf_diff']].count().rename(columns={'perf_diff': 'n'})
global_resulted_agg = global_resulted_agg_mean.merge(right=global_resulted_agg_std, left_index=True, right_index=True, how='outer')
global_resulted_agg = global_resulted_agg.merge(right=global_resulted_agg_count, left_index=True, right_index=True, how='outer')

def tester(x):
    arg = x['mean'] / (x['std'] / (x['n'] ** 0.5))
    pv = 1 - stats.t.cdf(x=arg, df=x['n'] - 1)
    return pv

global_resulted_agg['test_result'] = global_resulted_agg.apply(func=tester, axis=1)
global_resulted_agg = global_resulted_agg.sort_values(by='test_result')

cmp_alpha_thresh = 0.05

global_resulted_filtered = global_resulted_agg[global_resulted_agg['test_result'] <= cmp_alpha_thresh].copy()
if global_resulted_filtered.shape[0] > 0:
    the_chosen_one = ['{0}__{1}'.format(name, global_resulted_filtered.index.values[0])]
else:
    the_chosen_one = []


In [41]:
global_resulted_agg

,mean,std,n,test_result
transform,,,,
std3x6_div_pct,0.091281,0.163697,10,0.055836
std3x12_div_pct,0.114499,0.209613,10,0.059087
p101x3_div_pct,0.071378,0.183710,10,0.125177
min1x3_div_pct,0.066795,0.189675,10,0.147150
max3x12_div_pct,0.062714,0.180387,10,0.150065
max1x12_div_pct,0.017947,0.215969,10,0.399318
median1x3_div_pct,0.012882,0.166055,10,0.405857
min1x6_div_pct,0.015582,0.218291,10,0.413225
mean1x6_div_pct,0.007787,0.205296,10,0.453581


In [43]:
driver = 'p101x3_div_pct'
global_resulted[global_resulted['transform'] == driver]

,fold_n,transform,performed,stabilized,base_performed,perf_diff
29,0,p101x3_div_pct,0.054590,NaN,0.182068,-0.127478
29,1,p101x3_div_pct,0.123263,NaN,-0.063989,0.187252
29,2,p101x3_div_pct,0.137985,NaN,0.243518,-0.105533
29,3,p101x3_div_pct,0.275026,NaN,-0.070899,0.345925
29,4,p101x3_div_pct,0.179535,NaN,0.046473,0.133061
29,5,p101x3_div_pct,0.029974,NaN,0.179056,-0.149082
29,6,p101x3_div_pct,0.133682,NaN,-0.039337,0.173019
29,7,p101x3_div_pct,0.143519,NaN,0.210133,-0.066614
29,8,p101x3_div_pct,0.217820,NaN,0.207915,0.009904
29,9,p101x3_div_pct,0.358922,NaN,0.045592,0.313330
